In [4]:
import pandas as pd
import json
from haversine import haversine
import numpy as np
import datetime
import os
import glob
print(os.getcwd())
print(datetime.datetime.now())

/mnt/clients/juba/hqjubaapp02/sharefolder/TraderArea/BL_TA_request_X_miles
2020-12-29 23:17:54.444344


In [5]:
path_json_zip_centers="/mnt/clients/juba/hqjubaapp02/sharefolder/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json"
folder_store_list="/mnt/clients/juba/hqjubaapp02/sharefolder/biglots_data/static_files/Store_list/"


zip_centers=json.load(open(path_json_zip_centers,"r"))
latest_store_list=glob.glob(folder_store_list+"*.txt")
latest_store_list=sorted(latest_store_list,key=lambda x: os.stat(x).st_mtime)
latest_store_list=latest_store_list[-1]

In [7]:
dict_update_location_latlng={}

def revise_store_lat_lng(path_store_list_input=latest_store_list):
    df=pd.read_csv(path_store_list_input,sep="|",dtype=str)
    df['latitude_meas']=df['latitude_meas'].astype(float)
    df=df[~df['location_id'].isin(['145','6990'])]
    df=df[df['latitude_meas']==0]
    print("type in lat and lng: %d, %d")
    for i,row in df.iterrows():
        store_num=row['location_id']
        address=row['address_line_1']+", "+row['city_nm']+", "+row['state_nm']+", "+row['zip_cd']+", US"
        print(address)
        
        google_lat_long=str(input())
        lat=eval(google_lat_long)[0]
        lng=eval(google_lat_long)[1]
        dict_update_location_latlng.update({store_num:{"lat":lat,"lng":lng}})
    print("dict_update_location_latlng: \n %s"%dict_update_location_latlng)
        
revise_store_lat_lng()

type in lat and lng: %d, %d
4900 E DUBLIN GRANVILLE RD, COLUMBUS, OH, 43081-7651, US
40.0800688,-82.8604897
dict_update_location_latlng: 
 {'6991': {'lat': 40.0800688, 'lng': -82.8604897}}


In [8]:
def update_store_location(df):
    for store_num, store_revised_loc in dict_update_location_latlng.items():

        df.loc[df['location_id']==store_num,'latitude_meas']=store_revised_loc['lat']
        df.loc[df['location_id']==store_num,'longitude_meas']=store_revised_loc['lng']
    
    return df

In [15]:
df_store_location=pd.read_table(latest_store_list,sep="|",usecols=['location_id','latitude_meas','longitude_meas'])
df_store_location['location_id']=df_store_location['location_id'].astype(str)
df_store_location['latitude_meas']=df_store_location['latitude_meas'].astype(float)
df_store_location['longitude_meas']=df_store_location['longitude_meas'].astype(float)
df_store_location=update_store_location(df_store_location)
df_store_location=df_store_location[~df_store_location['location_id'].isin(["145","6990"])]
df_store_location=df_store_location.reset_index()
del df_store_location['index']

In [17]:
n_miles=10

df_output_zip_by_store=pd.DataFrame()
for i, row in df_store_location.iterrows():
    store_id=row['location_id']
    list_zip_10_around_store=[]
    list_dist_10_around_store=[]
    for zip_cd,z_center in zip_centers.items() :
        dist=haversine([row['latitude_meas'],row['longitude_meas']],z_center,unit="mi")
        if dist<=n_miles:
            list_zip_10_around_store.append(zip_cd)
            list_dist_10_around_store.append(dist)
    df=pd.DataFrame({"location_id":[store_id]*len(list_zip_10_around_store),
                     "zip_cd":list_zip_10_around_store,
                     "dist_mile":list_dist_10_around_store
                    },index=range(len(list_zip_10_around_store)))
    df_output_zip_by_store=df_output_zip_by_store.append(df)
print(df_output_zip_by_store.shape,df_output_zip_by_store['location_id'].nunique())          

(37080, 3) 1412


In [18]:
df_output_zip_by_store.head(2)

,location_id,zip_cd,dist_mile
0,3,30912,4.019458
1,3,30914,3.314609


In [19]:
df_store_location=pd.read_table(latest_store_list,sep="|")
df_store_location['location_id']=df_store_location['location_id'].astype(str)
df_store_location['latitude_meas']=df_store_location['latitude_meas'].astype(float)
df_store_location['longitude_meas']=df_store_location['longitude_meas'].astype(float)
df_store_location=update_store_location(df_store_location)
df_store_location=df_store_location[~df_store_location['location_id'].isin(["145","6990"])]
df_store_location=df_store_location.reset_index()
del df_store_location['index']

df_store_location.head(3)

,location_id,location_desc,open_dt,close_dt,address_line_1,address_line_2,city_nm,state_nm,zip_cd,longitude_meas,latitude_meas,sales_square_feet,furniture_sales_square_feet
0,3,PEACH ORCHARD PLAZA,1988-10-31,NaN,2708 PEACH ORCHARD RD,NaN,AUGUSTA,GA,30906-2408,-82.018921,33.421570,26501.0,5098.0
1,30,SOUTHWOOD PLZ S/C,1984-04-05,NaN,818 S MAIN ST,NaN,BOWLING GREEN,OH,43402-4601,-83.650909,41.363132,21322.0,3271.0
2,43,SOUTHLAND SHOPPING CENTER,1984-09-26,NaN,3410 GLENDALE AVE,NaN,TOLEDO,OH,43614-2428,-83.623582,41.612553,23541.0,3725.0


In [20]:
df_output_zip_all_unique=df_output_zip_by_store[['zip_cd']].drop_duplicates()

In [21]:
writer=pd.ExcelWriter("./BL_zips_in_10_miles_JL_%s.xlsx"%str(datetime.datetime.now().date()),engine="xlsxwriter")
df_output_zip_all_unique.to_excel(writer,"unique_zip_in_10_mi",index=False)
df_store_location.to_excel(writer,"store_info",index=False)
df_output_zip_by_store.to_excel(writer,"zip_by_store_in_10_mi",index=False)
writer.save()